# O que é a cadeia de Markov?

## Conceito Básico
Imagine que temos um conjunto de estados
𝑆, e cada estado representa uma possível situação em que o sistema pode estar. A cadeia de Markov descreve uma "caminhada" entre esses estados. Em um modelo de Markov de primeira ordem, a probabilidade de um estado futuro depende apenas do estado atual e não dos anteriores. Isso simplifica bastante o cálculo de probabilidades ao longo do tempo.


## Exemplo Prático: Gerador de Texto
Imagine que queremos gerar frases com base em um texto. Com a cadeia de Markov, podemos pegar uma frase e contar a frequência das transições entre as palavras.

Suponha que você tem a frase `"o gato dorme no tapete"`. Podemos fazer o seguinte mapeamento de transição de palavras:

> "o" → "gato"
>
> "gato" → "dorme"
>
> "dorme" → "no"
>
> "no" → "tapete"

Depois de processar várias frases, construímos um modelo de probabilidade que diz que, por exemplo, a palavra "o" tem uma alta chance de ser seguida por "gato" e que "dorme" é seguido por "no".

Se quisermos gerar uma frase, podemos escolher uma palavra inicial (ex., "o") e seguir as transições prováveis para criar uma sequência como: "o gato dorme no tapete".


## Exemplo de Frases de Entrada

As frases fornecidas para construir a cadeia de Markov são:

1. `Café ajuda na concentração`
2. `Na hora de programar, ajuda um café`
3. `Ajuda eu a programar pegando um café`

Essas frases são separadas em pares de palavras (estado atual e próxima palavra) para definir transições de estado. As probabilidades de transição são calculadas com base na frequência de cada transição observada nas frases.

### Tabela de Transições e Probabilidades

| Estado                     | Próxima Palavra | Frequência | Probabilidade (%) |
|----------------------------|-----------------|------------|-------------------|
| Café                       | ajuda           | 1          | 100%              |
| ajuda                      | na, um          | 1, 1       | 50%, 50%          |
| Ajuda                      | eu              | 1          | 100%              |


## Como Funciona

1. **Construção da Cadeia**: As frases de entrada são analisadas e divididas em pares de palavras consecutivas.
2. **Probabilidades de Transição**: A probabilidade de cada transição é calculada dividindo a frequência de cada par pelo total de ocorrências do estado inicial.
3. **Geração de Frases**: Para gerar uma nova frase, o algoritmo escolhe um estado inicial e, com base nas probabilidades, segue as transições até alcançar um estado final.

## Exemplo de Geração de Frase

Suponha que o estado inicial seja `Café ajuda`:
- Com 100% de probabilidade, a próxima palavra será `na`.
- Em seguida, de `ajuda na`, a transição para `concentração` tem 100% de probabilidade.
- Assim, o algoritmo gera a frase `Café ajuda na concentração`.

Esse processo pode ser repetido para gerar novas frases que sigam os padrões observados.

In [40]:
import random
from collections import defaultdict

def construir_modelo_markov(frases):
  """
  Constrói um modelo de Markov a partir de um vetor de frases.

  Parameters
  ----------
  frases : list
      Uma lista de frases para construir o modelo.

  Returns
  ----------
  modelo : dict
      Um dicionário que mapeia palavras a listas de palavras seguintes.
  """

  modelo = defaultdict(list)
  for frase in frases:
      palavras = frase.split()
      for i in range(len(palavras) - 1):
          modelo[palavras[i]].append(palavras[i + 1])
  return modelo

def gerar_texto(modelo, palavra_inicial, tamanho=10):
  """
  Gera um texto com base no modelo de Markov, passando, o modelo, a palavra inicial e o tamanho desejado.

  Parameters
  ----------
  modelo : dict
      Um dicionário que mapeia palavras a listas de palavras seguintes.
  palavra_inicial : str
      A palavra a partir da qual o texto será gerado.
  tamanho : int, opcional
      O número de palavras a serem geradas. O padrão é 10.

  Returns
  ----------
      Um texto gerado com base no modelo de Markov.
  """

  palavra_atual = palavra_inicial
  resultado = [palavra_atual]

  for _ in range(tamanho - 1):
      if palavra_atual not in modelo or not modelo[palavra_atual]:
          break  # Interrompe se não houver próxima palavra
      proxima_palavra = random.choice(modelo[palavra_atual])
      resultado.append(proxima_palavra)
      palavra_atual = proxima_palavra
  return ' '.join(resultado)

In [41]:
# Exemplo de uso com um vetor de frases
frases_exemplo = [
    "o gato dorme no tapete",
    "o cachorro late na rua",
    "a gata observa o cachorro",
    "a gata está miando",
    "o tapete é macio",
    "a rua está movimentada",
    "o gato pula no sofá"
]

# printa cada palavra e as possibilidades qjue ela tem
modelo_markov = construir_modelo_markov(frases_exemplo)
for palavra, proximo_palavras in modelo_markov.items():
    print(f"{palavra}: {proximo_palavras}")

o: ['gato', 'cachorro', 'cachorro', 'tapete', 'gato']
gato: ['dorme', 'pula']
dorme: ['no']
no: ['tapete', 'sofá']
cachorro: ['late']
late: ['na']
na: ['rua']
a: ['gata', 'gata', 'rua']
gata: ['observa', 'está']
observa: ['o']
está: ['miando', 'movimentada']
tapete: ['é']
é: ['macio']
rua: ['está']
pula: ['no']


In [42]:
print(gerar_texto(modelo_markov, "gata", 5))

gata está movimentada


# Como podemos melhorar o modelo?

- Considerar de outra maneira "ajuda" e "Ajuda"
- Considerar trazer % para considerar a decisão
- Quebrar palavras em pedaçõs menores ainda, como exemplo por exemplo, **gat** poderia ser:
  - gata
  - gato
  - gateway
  - gatuno
  - gatuna
  - ...

## Tentando mudar para porcentagem e ver o comportamento

Antes pegavamos o primeiro das opções, podemos pegar com %.

In [43]:
from collections import defaultdict, Counter
import random

def construir_modelo_markov_porcentagem(frases):
  modelo = construir_modelo_markov(frases)

  # coloca a porcentagem com base na qtd de palavras no dict
  for palavra, proximo_palavras in modelo.items():
      total_proximos = len(proximo_palavras)
      # Count occurrences of each next word
      contagem_proximos = Counter(proximo_palavras)
      # Calculate percentage for each next word based on its count
      modelo[palavra] = {proxima: (count / total_proximos) * 100 for proxima, count in contagem_proximos.items()}

  return modelo

for palavra, proximo_palavras in construir_modelo_markov_porcentagem(frases_exemplo).items():
    print(f"{palavra}: {proximo_palavras}")

o: {'gato': 40.0, 'cachorro': 40.0, 'tapete': 20.0}
gato: {'dorme': 50.0, 'pula': 50.0}
dorme: {'no': 100.0}
no: {'tapete': 50.0, 'sofá': 50.0}
cachorro: {'late': 100.0}
late: {'na': 100.0}
na: {'rua': 100.0}
a: {'gata': 66.66666666666666, 'rua': 33.33333333333333}
gata: {'observa': 50.0, 'está': 50.0}
observa: {'o': 100.0}
está: {'miando': 50.0, 'movimentada': 50.0}
tapete: {'é': 100.0}
é: {'macio': 100.0}
rua: {'está': 100.0}
pula: {'no': 100.0}


In [44]:
def gerar_texto_porcentagem(modelo, palavra_inicial, tamanho=10):
  modelo = construir_modelo_markov_porcentagem(frases_exemplo)
  palavra_atual = palavra_inicial
  resultado = [palavra_atual]

  for _ in range(tamanho):
      if palavra_inicial not in modelo:
          break
      proximos = modelo[palavra_inicial]
      proxima_palavra = random.choices(list(proximos.keys()), weights=list(proximos.values()))[0]

      print(proxima_palavra, end=' ')
      palavra_inicial = proxima_palavra

      resultado.append(proxima_palavra)

  return ' '.join(resultado)


In [46]:
inicio = "gata"

for i in range(4):
  print(inicio, end=' ')
  print(gerar_texto_porcentagem(modelo_markov, inicio, 5))

gata observa o cachorro late na gata observa o cachorro late na
gata observa o cachorro late na gata observa o cachorro late na
gata está movimentada gata está movimentada
gata está miando gata está miando


# O que podemos observar?

## **Antes**
- Modelo gerar a mesma frase sempre, ajudava muito, mas só com as mesmas X palavras...
- Sempre pegava a mesma palavra

## **Depois**
- Modelo gera frases diferentes (pois pega o que tem a maior probabilidade, podendo gerar frases mais coesas)
- Pega a palavra que mais foi usada nas frases que ele treinou
- **PROBLEMA**: pode cair em um erro de ficar em loop (pois é tipo um grafo)